# Demo Lakehouse

El presente demo muestra a nivel conceptual como funciona una arquitectura Bronze-Silver-Gold.

En el demo, las capas están divididas de la siguiente forma:

* **Bronze**  : ingesta cruda, en este caso es de archivos CSV - normalmente vienen de un sistema externo
* **Silver**  : transformaciones de datos
* **Gold**    : clasificación + agregaciones pensando en las vistas que utilizarían los usuarios finales.

Dataset a utilizar: *winequality-red.csv*

In [0]:
# El dataset de Wine Quality está disponible en Databricks dentro de los datasets de ejemplo
file = dbutils.fs.ls("/databricks-datasets/wine-quality/winequality-red.csv")

# Capa Bronze

In [0]:
df_bronze = (spark.read
             .option("header", True)
             .option("inferSchema", True)   
             .option("sep",";")
             .csv(file[0].path))
df_bronze.printSchema()
df_bronze.head(5)
display(df_bronze)

Transformación *"liviana"* - el espacio ` ` es un caracter ilegal en la tablas, entonces reemplazamos el espacio por un `_`

In [0]:
from pyspark.sql.functions import col
df_bronze = df_bronze.select([col(column).alias(column.replace(' ', '_')) for column in df_bronze.columns])

Y lo salvamos como una tabla en el catalogo

In [0]:
(df_bronze
 .write
 .format("delta")
 .mode("overwrite")
 .saveAsTable(f"big_data_ii_2025.spark_examples.wine_bronze"))